<a href="https://colab.research.google.com/github/GretaDanesi18/Olive_Detection_Tracking/blob/main/target_detector/YOLOv5_TDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP**

Per prima cosa importiamo le librerie necessarie

In [1]:
import torch 
import os
import random 
import shutil
import glob
from IPython.display import Image,display
from PIL import Image, ImageDraw
import cv2
import pandas as pd
import subprocess
from numpy import source


Controlliamo l'utilizzo del device GPU oppure CPU

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.12.1+cu113  Device: cuda


Successivamente cloniamo la repo contente YOLOv5 e installiamo i requisiti

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

##**CREAZIONE DATASET TARGET DETECTOR**

Caricare la cartella contenente le pseudo labels generate tramite i software contenuti nella seguente repo:
https://github.com/Lio320/GraphTracking.git

Possiamo caricarle in diversi modi:


* in locale 
```
from google.colab import files
uploaded = files.upload()
```
*  con google drive
```
from google.colab import drive
drive.mount('/content/gdrive')
```
* con github

 ```
 !git clone "inserire url della repo in cui si trovano le pseudo labels"
 ```

In [ ]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

Creiamo le cartelle per il dataset del target detector.

La struttura del dataset del target detector è la stessa del source detector con l'aggiunta delle cartelle per le labels Sfm e features




```
/path/to/dataset/...
Sfm
  +--images
  |   +---train
         +-----image_1.jpg
         +-----image_2.jpg
         +-----image_n.jpg
  |   +---val
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg
  +--labels
  |   +---train
         +-----label_1.txt
         +-----label_2.txt
         +-----label_n.txt
  |   +---val
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt
features
  +--images
  |   +---train
        +-----image_1.jpg
         +-----image_2.jpg
         +-----image_n.jpg
  |   +---val
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg
  +--labels
  |   +---train
         +-----label_1.txt
         +-----label_2.txt
         +-----label_n.txt
  |   +---val
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt


+---test
  +--labels
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt
  +--images
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg

```


In [5]:
try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/images/train')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/features/images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/images/val')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/features/images/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/images/train')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/Sfm/images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/images/val')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/Sfm/images/val')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/labels/train')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/features/labels/train')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/labels/val')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/features/labels/val')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/labels/train')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/Sfm/labels/train')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/labels/val')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/Sfm/labels/val')


Faccio lo split del target dataset cosi da ottenere il dataset per il training e per la validation

In [6]:
#path source dataset
source_images_path = '/content/yolov5/Olive_Detection_Tracking/source_detector/dataset_source/images'
source_labels_path = '/content/yolov5/Olive_Detection_Tracking/source_detector/dataset_source/labels'

#path Sfm
Sfm_images_path = '/content/yolov5/Olive_Detection_Tracking/target_detector/pseudo_labels_dataset/images'
Sfm_labels_path = '/content/yolov5/Olive_Detection_Tracking/target_detector/pseudo_labels_dataset/SfmLabels/results_2/labels'
Sfm_train_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/images/train'
Sfm_val_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/images/val'
Sfm_train_label_path ='/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/labels/train'
Sfm_val_label_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/Sfm/labels/val'

#path features
features_images_path = '/content/yolov5/Olive_Detection_Tracking/target_detector/pseudo_labels_dataset/images'
features_labels_path = '/content/yolov5/Olive_Detection_Tracking/target_detector/pseudo_labels_dataset/features_label/results_2/labels'
features_train_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/images/train'
features_val_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/images/val'
features_train_label_path ='/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/labels/train'
features_val_label_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/features/labels/val'

#percentuale per train, val
Sfm_val_ratio = 0.2
Sfm_train_ratio = 0.8

features_val_ratio = 0.2
features_train_ratio = 0.8

Sfm_lista_label = sorted(os.listdir(Sfm_labels_path) + os.listdir(source_labels_path))
Sfm_lista_image = sorted(os.listdir(Sfm_images_path) + os.listdir(source_images_path))
features_lista_label = sorted(os.listdir(features_labels_path) + os.listdir(source_labels_path))
features_lista_image = sorted(os.listdir(features_images_path) + os.listdir(source_images_path))


In [7]:
#numero di immagini Sfm per train e per val
Sfm_num_train = int(len(Sfm_lista_image)*Sfm_train_ratio)
Sfm_num_val = int(len(Sfm_lista_image)*Sfm_val_ratio)

print("Immagini per training: ", Sfm_num_train)
print("Immagini per validation: ", Sfm_num_val)

Immagini per training:  383
Immagini per validation:  95


In [8]:
#numero di immagini features per train e per val
features_num_train = int(len(features_lista_image)*features_train_ratio)
features_num_val = int(len(features_lista_image)*features_val_ratio)

print("Immagini per training: ", features_num_train)
print("Immagini per validation: ", features_num_val)

Immagini per training:  383
Immagini per validation:  95


In [9]:
#inserisco Sfm image e label in train
for i in range(Sfm_num_train):
   file_image = random.choice(Sfm_lista_image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   if os.path.exists(Sfm_labels_path+'/'+file_label):
    source_label = Sfm_labels_path+'/'+file_label

   elif os.path.exists(source_labels_path+'/'+file_label):
    source_label = source_labels_path+'/'+file_label

   destination_label = Sfm_train_label_path+'/'+file_label
   shutil.copyfile(source_label,destination_label)

   if os.path.exists(Sfm_images_path+'/'+file_image):
    source_image = Sfm_images_path+'/'+file_image

   elif os.path.exists(source_images_path+'/'+file_image):
    source_image = source_images_path+'/'+file_image

   destination_image = Sfm_train_image_path+'/'+file_image
   shutil.copyfile(source_image,destination_image)

   #rimuovo i file dalla lista
   Sfm_lista_image.remove(file_image)
   Sfm_lista_label.remove(file_label)
    

#inserisco Sfm image e label in val  
for i in range(Sfm_num_val):
    file_image =random.choice(Sfm_lista_image)
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #sposto i file nella cartella val
    if os.path.exists(Sfm_labels_path+'/'+file_label):
     source_label = Sfm_labels_path+'/'+file_label

    elif os.path.exists(source_labels_path+'/'+file_label):
     source_label = source_labels_path+'/'+file_label

    destination_label = Sfm_val_label_path+'/'+file_label
    shutil.copyfile(source_label,destination_label)

    if os.path.exists(Sfm_images_path+'/'+file_image):
     source_image = Sfm_images_path+'/'+file_image

    elif os.path.exists(source_images_path+'/'+file_image):
     source_image = source_images_path+'/'+file_image

    destination_image = Sfm_val_image_path+'/'+file_image
    shutil.copyfile(source_image,destination_image)


    #rimuovo i file dalla lista
    Sfm_lista_image.remove(file_image)
    Sfm_lista_label.remove(file_label)

In [10]:
#inserisco features image e label in train
for i in range(features_num_train):
   file_image = random.choice(features_lista_image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   if os.path.exists(features_labels_path+'/'+file_label):
    source_label = features_labels_path+'/'+file_label

   elif os.path.exists(source_labels_path+'/'+file_label):
    source_label = source_labels_path+'/'+file_label

   destination_label = features_train_label_path+'/'+file_label
   shutil.copyfile(source_label,destination_label)

   if os.path.exists(features_images_path+'/'+file_image):
    source_image = features_images_path+'/'+file_image

   elif os.path.exists(source_images_path+'/'+file_image):
    source_image = source_images_path+'/'+file_image

   destination_image = features_train_image_path+'/'+file_image
   shutil.copyfile(source_image,destination_image)


   #rimuovo i file dalla lista
   features_lista_image.remove(file_image)
   features_lista_label.remove(file_label)


    

#inserisco features image e label in val  
for i in range(features_num_val):
    file_image =random.choice(features_lista_image)
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #sposto i file nella cartella val
    if os.path.exists(features_labels_path+'/'+file_label):
     source_label = features_labels_path+'/'+file_label

    elif os.path.exists(source_labels_path+'/'+file_label):
     source_label = source_labels_path+'/'+file_label

    destination_label = features_val_label_path+'/'+file_label
    shutil.copyfile(source_label,destination_label)

    if os.path.exists(features_images_path+'/'+file_image):
     source_image = features_images_path+'/'+file_image

    elif os.path.exists(source_images_path+'/'+file_image):
     source_image = source_images_path+'/'+file_image

    destination_image = features_val_image_path+'/'+file_image
    shutil.copyfile(source_image,destination_image)


    #rimuovo i file dalla lista
    features_lista_image.remove(file_image)
    features_lista_label.remove(file_label)

Cambiare i path all' interno del file target_detector_features.yaml in accordo con il proprio dataset

In [ ]:
#inserisco target_detector_features.yalm nella directory 'data'
source = 'Olive_Detection_Tracking/target_detector/target_detector_features.yaml'
destination = 'data/target_detector_features.yaml'
shutil.copy(source,destination)

Cambiare i path all' interno del file target_detector_Sfm.yaml in accordo con il proprio dataset

In [ ]:
#inserisco target_detector_Sfm.yal nella directory 'data'
source = 'Olive_Detection_Tracking/target_detector/target_detector_Sfm.yaml'
destination = 'data/target_detector_Sfm.yaml'
shutil.copy(source,destination)

## **ADDESTRAMENTO TARGET DETECTOR**

Usiamo Weights & Biases per mostrare i risultati.

Eventualmente possiamo usare anche tensorboard lanciando il seguente codice dopo aver avviato il training.

```
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs
```



In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()

Avviamo l'addestramento

In [ ]:
!python train.py --img 416 --batch 16 --epochs 80 --data target_detector_features.yaml --weights yolov5s.pt --cache 

In [ ]:
!python train.py --img 416 --batch 16 --epochs 80 --data target_detector_Sfm.yaml --weights yolov5s.pt --cache 

Facciamo il download del weights migliore cosi da salvarlo in locale e poterlo riutilizzare per inferences e validation future

Modificare i path in accordo con le proprie directory


In [ ]:
from google.colab import files
files.download('runs/train/exp/weights/best.pt')

## **TEST TARGET DETECTOR**

Dopo aver fatto il training facciamo il test sul dateset apposito


In [ ]:
!python val.py --img 1280 --batch 16 --data target_detector_features.yaml --weights runs/train/exp/weights/best.pt --task test --save-txt 

In [ ]:
!python val.py --img 1280 --batch 16 --data target_detector_Sfm.yaml --weights runs/train/exp2/weights/best.pt --task test --save-txt 

Salvo i risultati

Modificare i path in base alle proprie directory

In [ ]:
!zip -r validation_features_results.zip runs/val/exp/

In [ ]:
from google.colab import files
files.download("validation_features_results.zip")

In [ ]:
!zip -r validation_sfm_results.zip runs/val/exp2/

In [ ]:
from google.colab import files
files.download("validation_sfm_results.zip")

## **DETECTION TARGET DETECTOR**

Infine faccio la detection su un nuovo set di immagini creato estraendo i frame dal video2 che si trova al seguente path Olive_Detection_Tracking/dataset_video/videos/

Per salvare sia le labels che le immagini facciamo il download di tutta la cartella in cui sono salvati i risultati runs/detect/exp

Modificare i path in base alle proprie directory

In [ ]:
!python detect.py --source Olive_Detection_Tracking/dataset_video/frame_video/frame_video2 --data data/target_detector_features.yaml --weights runs/train/exp/weights/best.pt --save-txt --conf-thres 0.40

In [ ]:
!zip -r target_detection_features_results.zip runs/detect/exp/

In [ ]:
from google.colab import files
files.download("target_detection_features_results.zip")

In [ ]:
!python detect.py --source Olive_Detection_Tracking/dataset_video/frame_video/frame_video2 --data data/target_detector_Sfm.yaml --weights runs/train/exp2/weights/best.pt --save-txt --conf-thres 0.40

In [ ]:
!zip -r target_detection_sfm_results_finale.zip runs/detect/exp2/

In [ ]:
from google.colab import files
files.download("target_detection_sfm_results.zip")


In [ ]:
!zip -r risultati.zip runs/

Per fare il download di tutti i risultati 
 ```
 !zip -r risultati.zip runs/
 from google colab import files
 files.download("risultati.zip")
 ```
Se salvare i dati in locale impiega troppo tempo conviene usare il drive 

```
from google.colab import drive
drive.mount('/content/gdrive')
shutil.move(source, destination)
```
